In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

D:\Python\Projects\Assistance\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True).to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=0 if device == "cuda" else -1,
)

Device set to use cpu


In [3]:
# from ollama import chat
# from ollama import ChatResponse
#
# class Llama_3b:
#
#     def get_response(self, prompt=str):
#         response: ChatResponse = chat(model='llama3.2:3b', messages=[
#           {
#             'role': 'user',
#             'content': prompt,
#           },
#         ])
#         return response.message.content
# model = Llama_3b()
# print(model.get_response('what is the happiest country in the world'))

In [4]:
import pyaudio
import wave
def get_audio():
    # Parameters
    FORMAT = pyaudio.paInt16  # Audio format
    CHANNELS = 1  # Mono
    RATE = 44100  # Sample rate
    CHUNK = 1024  # Chunk size
    RECORD_SECONDS = 5  # Duration of recording
    OUTPUT_FILENAME = "output.wav"  # Output file name

    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Open stream
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    # Record audio
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded audio to a file
    with wave.open(OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Audio saved to '{OUTPUT_FILENAME}'.")

In [6]:
def transcribe(audio):
    result = pipe(audio)
    return result['text']
transcribe('output.wav')

ValueError: ffmpeg was not found but is required to load audio files from filename